## Validation on titanic dataset

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/CreateTool-AGI/blob/main/examples/titanic.ipynb)

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/CreateTool-AGI.git

In [2]:
from createtoolagi import CTAGI

#import os
#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

import pkg_resources
package_name = "createtoolagi"  
pkg = pkg_resources.get_distribution(package_name)
print(pkg.version)

1.1.1


## Prepare training data

In [12]:
import csv

data_set = ''
with open('../data/learning.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        data_set += ','.join(row) + '\n'


In [13]:
training_data = """
{data_set_}
------------------
・Given the above data, predict "survived" using "pclass,sex,age,sibsp,arch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone" 
as input values between 0 and 1. 
・Predict the value of "survived" between 0 and 1. 
・Please write "used for prediction.The input is a comma-separated string.For example, it receives inputs such as '2,male,27.0,0,0,13.0,S,Second,man,True,,Southampton,no,True'." in the description of the code to be created.
・The input data should be character strings as follows.
------------------
2,male,52.0,0,0,13.0,S,Second,man,True,,Southampton,no,True
""".format(data_set_ = data_set)

In [14]:
validation_data = "Predict 'survived' based on the following data.2,male,27.0,0,0,13.0,S,Second,man,True,,Southampton,no,True"

## Normal GPT-4

In [21]:
from langchain.llms import OpenAI

gpt4 = OpenAI(temperature=0, model_name = 'gpt-4')

In [22]:
gpt4(input)

'0'

## CreateTool-AGI

In [237]:
!rm -r ../tools/titanic

In [6]:
from createtoolagi import CTAGI

from qdrant_client import QdrantClient
from qdrant_client import models

base_model_name = 'gpt-4'
#base_model_name = 'gpt-3.5-turbo'
create_model_name = 'gpt-4'
embegging_model_name = 'text-embedding-ada-002'

qdrant = QdrantClient(path='../tools/titanic')
qdrant.create_collection(collection_name ="tool_store", vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE))

ctagi = CTAGI(
    base_model_name=base_model_name, 
    create_model_name=create_model_name, 
    embegging_model_name=embegging_model_name,
    qdrant=qdrant
    )

In [16]:
ctagi.run(training_data = training_data, validation_data = validation_data)

> Search tools.
Search results：
[ScoredPoint(id=1, version=0, score=0.7945618846752237, payload={'description': 'Predicts survival based on input data. The input is a string with values separated by commas.', 'code': 'from langchain.tools import BaseTool\n\nclass SurvivalPredictionTool(BaseTool):\n    name = "SurvivalPredictionTool"\n    description = "Predicts survival based on input data. The input is a string with values separated by commas."\n\n    def _run(self, query: str) -> float:\n        "Use the tool."\n        pclass, sex, age, sibsp, parch, fare, embarked, _class, who, adult_male, deck, embark_town, alive, alone = query.split(",")\n\n        # Convert string values to numerical values\n        pclass = int(pclass)\n        sex = 1 if sex == "male" else 0\n        age = float(age) if age else 0\n        sibsp = int(sibsp)\n        parch = int(parch)\n        fare = float(fare)\n        embarked = {"S": 0, "C": 1, "Q": 2}.get(embarked, 0)\n        _class = {"First": 1, "Seco

## Predict

In [ ]:
pred_data = "3,male,32.0,0,0,7.75,Q,Third,man,True,,Queenstown,no,True"

input = """
Please predict the following titanic passengers.
------------------
pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
{pred_data_}
""".format(pred_data_ = pred_data)


In [ ]:
ctagi.run(input=input)

## Others

In [8]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent

In [185]:
from langchain.tools import BaseTool

class MultiplicationTool(BaseTool):
    name = "SurvivalPredictionTool"
    description = "used for prediction.The input is a comma-separated string."

    def _run(self, query: str) -> str:
        "Use the tool."
        a, b = query.split(",")
        c = int(a) * int(b)
        result = c
        return result 

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("MultiplicationTool does not support async")  

In [186]:
tools =[]

MultiplicationTool = MultiplicationTool()
tools.append(MultiplicationTool)

tools

[MultiplicationTool(name='SurvivalPredictionTool', description='used for prediction.The input is a comma-separated string.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None)]

In [151]:
validation_data = "7 times 8"

In [152]:
class SurvivalPredictionTool(BaseTool):
    name = "SurvivalPredictionTool"
    description = "used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."
    #name = "SurvivalPredictionTool"
    #description = "It is useful for predicting 'survived'"

    def _run(self, query: str) -> str:
        "Use the tool."
        return query

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("MultiplicationTool does not support async") 

In [226]:

from langchain.tools import BaseTool

class SurvivalPredictionTool(BaseTool):
    name = "SurvivalPredictionTool"
    description = "used for prediction.The input is a comma-separated string.For example, it receives inputs such as '2,male,27.0,0,0,13.0,S,Second,man,True,,Southampton,no,True'."

    def _run(self, query: str) -> str:
        "Use the tool."
        pclass, sex, age, sibsp, parch, fare, embarked, _class, who, adult_male, deck, embark_town, alive, alone = query.split(",")

        # Convert string values to numerical values
        pclass = int(pclass)
        sex = 1 if sex == "male" else 0
        age = float(age) if age else 0
        sibsp = int(sibsp)
        parch = int(parch)
        fare = float(fare)
        embarked = {"S": 0, "C": 1, "Q": 2}.get(embarked, 0)
        adult_male = 1 if adult_male == "True" else 0
        alone = 1 if alone == "True" else 0

        # Calculate survival score based on input values
        survival_score = 0
        survival_score += (1 - pclass / 3) * 0.3
        survival_score += (1 - sex) * 0.3
        survival_score += (1 - age / 100) * 0.1
        survival_score += (1 - sibsp / 10) * 0.05
        survival_score += (1 - parch / 10) * 0.05
        survival_score += (1 - fare / 100) * 0.1
        survival_score += (1 - embarked / 2) * 0.05
        survival_score += (1 - adult_male) * 0.3
        survival_score += (1 - alone) * 0.05

        # Normalize survival score between 0 and 1
        survival_score = min(max(survival_score, 0), 1)

        return survival_score

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("SurvivalPredictionTool does not support async")


In [227]:
tools =[]

SurvivalPredictionTool = SurvivalPredictionTool()
tools.append(SurvivalPredictionTool)

tools

[SurvivalPredictionTool(name='SurvivalPredictionTool', description="used for prediction.The input is a comma-separated string.For example, it receives inputs such as '2,male,27.0,0,0,13.0,S,Second,man,True,,Southampton,no,True'.", args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None)]

In [228]:
validation_data = "Please use the following character string as input data.'2,male,27.0,0,0,13.0,S,Second,man,True,,Southampton,no,True'"

In [229]:
gpt4 = OpenAI(temperature=0, model_name = 'gpt-4')

agent = initialize_agent(
            tools = tools, 
            llm = gpt4, 
            agent="zero-shot-react-description", 
            verbose=True
            )


In [230]:
agent.run(validation_data)



> Entering new AgentExecutor chain...
I need to use the SurvivalPredictionTool to predict the survival of this individual.
Action: SurvivalPredictionTool
Action Input: 2,male,27.0,0,0,13.0,S,Second,man,True,,Southampton,no,True
Observation: 0.41
Thought:I now know the final answer.
Final Answer: The predicted survival probability for this individual is 0.41.

> Finished chain.


'The predicted survival probability for this individual is 0.41.'

In [218]:
from langchain.tools import BaseTool

class MultiplicationTool(BaseTool):
    name = "MultiplicationTool"
    description = "used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."

    def _run(self, query: str) -> str:
        "Use the tool."
        a, b = query.split(",")
        c = int(a) * int(b)
        result = c
        return result 

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("MultiplicationTool does not support async")

In [219]:
tools =[]

MultiplicationTool = MultiplicationTool()
tools.append(MultiplicationTool)

tools

[MultiplicationTool(name='MultiplicationTool', description="used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'.", args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None)]

In [223]:
validation_data = "8 times 7"

In [224]:
gpt4 = OpenAI(temperature=0, model_name = 'gpt-4')

agent = initialize_agent(
            tools = tools, 
            llm = gpt4, 
            agent="zero-shot-react-description", 
            verbose=True
            )

In [225]:
agent.run(validation_data)



> Entering new AgentExecutor chain...
I need to multiply 8 by 7
Action: MultiplicationTool
Action Input: 8,7
Observation: 56
Thought:I now know the final answer
Final Answer: 56

> Finished chain.


'56'